In [1]:
import datetime
import random

import numpy as np
from keras.layers import Activation
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dense
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.applications.resnet50 import ResNet50
from keras.applications import VGG16
from keras.applications.inception_v3 import InceptionV3
from tqdm import tqdm

from common import *

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
print(".. setting base configuration & params ..")

initialize()

best_model_filepath = './saved-models/achitecture-02.hdf5'
training_epochs = 20
batch_size = 10
images_size = 270


.. setting base configuration & params ..


In [7]:
(train_tensors, X_train, y_train,
test_tensors, X_test, y_test, 
valid_tensors, X_validate, y_validate, 
duration_loading) = load_and_split_data(images_size=images_size)

  1%|          | 9/1408 [00:00<00:17, 79.51it/s]

.. loading & splitting data ..


100%|██████████| 352/352 [00:03<00:00, 111.12it/s]


In [8]:
print(".. constructing the model ..")

model = Sequential()

model.add(Conv2D(filters=16, kernel_size=2, padding='same', input_shape=(images_size, images_size, 3), 
                 data_format="channels_last"))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=2, padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(GlobalAveragePooling2D())
model.add(Dense(2, activation='softmax'))

model.summary()

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


.. constructing the model ..
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 270, 270, 16)      208       
_________________________________________________________________
batch_normalization_5 (Batch (None, 270, 270, 16)      64        
_________________________________________________________________
activation_5 (Activation)    (None, 270, 270, 16)      0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 135, 135, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 135, 135, 32)      2080      
_________________________________________________________________
batch_normalization_6 (Batch (None, 135, 135, 32)      128       
_________________________________________________________________
activation_6 (Activation)    (None, 135, 135, 3

In [9]:
duration_training = train_single_model(model, 
                                       best_model_filepath, 
                                       train_tensors, y_train, 
                                       valid_tensors, y_validate, 
                                       training_epochs, 
                                       batch_size)


.. training the model ..
Train on 1408 samples, validate on 352 samples
Epoch 1/20
1408/1408 [==============================] - 418s 297ms/step - loss: 0.3922 - acc: 0.8203 - val_loss: 0.5018 - val_acc: 0.7869

Epoch 00001: val_loss improved from inf to 0.50176, saving model to ./saved-models/achitecture-02.hdf5
Epoch 2/20
1408/1408 [==============================] - 431s 306ms/step - loss: 0.3152 - acc: 0.8509 - val_loss: 0.9164 - val_acc: 0.8125

Epoch 00002: val_loss did not improve from 0.50176
Epoch 3/20
1408/1408 [==============================] - 269s 191ms/step - loss: 0.2880 - acc: 0.8707 - val_loss: 1.0046 - val_acc: 0.8153

Epoch 00003: val_loss did not improve from 0.50176
Epoch 4/20
1408/1408 [==============================] - 256s 182ms/step - loss: 0.2705 - acc: 0.8693 - val_loss: 0.2825 - val_acc: 0.8864

Epoch 00004: val_loss improved from 0.50176 to 0.28250, saving model to ./saved-models/achitecture-02.hdf5
Epoch 5/20
1408/1408 [==============================] - 268s

In [10]:
print(".. loading best weights ..")

model.load_weights(best_model_filepath)

.. loading best weights ..


In [11]:
print(".. testing the model on Test data ..")
section_start_time = datetime.datetime.utcnow()

test_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(test_tensors)]

test_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 0/440 [00:00<?, ?it/s]

.. testing the model on Test data ..


100%|██████████| 440/440 [00:40<00:00, 11.33it/s]


In [12]:
print(".. testing the model on Training data ..")
section_start_time = datetime.datetime.utcnow()

train_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(train_tensors)]

train_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/1408 [00:00<03:02,  7.72it/s]

.. testing the model on Training data ..


100%|██████████| 1408/1408 [02:04<00:00, 11.29it/s]


In [13]:
print(".. testing the model on Validation data ..")
section_start_time = datetime.datetime.utcnow()

validation_data_predictions = [np.argmax(model.predict(np.expand_dims(tensor, axis=0))) for tensor in tqdm(valid_tensors)]

validation_data_predictions_time = (datetime.datetime.utcnow() - section_start_time).total_seconds()

  0%|          | 1/352 [00:00<00:49,  7.08it/s]

.. testing the model on Validation data ..


100%|██████████| 352/352 [00:31<00:00, 11.30it/s]


In [14]:
print("________________________________________________________")
print("TIMING SUMMARY:\n")
print("loading duration: {0:.1f} seconds".format(duration_loading))
print("training duration: {0:.1f} seconds".format(duration_training))

print("________________________________________________________")
print("MODEL PERFORMANCE ON TEST DATA:\n")
print("predict duration: {0:.1f} seconds".format(test_data_predictions_time))
false_positive_images, false_negative_images = summarize_model_performance(X_test, y_test, test_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON TRAIN DATA:\n")
print("predict duration: {0:.1f} seconds".format(train_data_predictions_time))
_, _ = summarize_model_performance(X_train, y_train, train_data_predictions)

print("________________________________________________________")
print("MODEL PERFORMANCE ON VALIDATION DATA:\n")
print("predict duration: {0:.1f} seconds".format(validation_data_predictions_time))
_, _ = summarize_model_performance(X_validate, y_validate, validation_data_predictions)

________________________________________________________
TIMING SUMMARY:

loading duration: 26.1 seconds
training duration: 8913.1 seconds
________________________________________________________
MODEL PERFORMANCE ON TEST DATA:

predict duration: 40.5 seconds
all:  440
all_positives:  81
all_negatives:  359
true_positives:  71
true_negatives:  343
false_positives:  16
false_negatives:  10
RECALL: 0.88
SPECIFICITY: 0.96
ACCURACY: 0.94
PRECISION: 0.82
F1 SCORE: 0.85
FP RATE / ERROR I: 0.04
FN RATE / ERROR II: 0.12
________________________________________________________
MODEL PERFORMANCE ON TRAIN DATA:

predict duration: 124.7 seconds
all:  1408
all_positives:  316
all_negatives:  1092
true_positives:  255
true_negatives:  1047
false_positives:  45
false_negatives:  61
RECALL: 0.81
SPECIFICITY: 0.96
ACCURACY: 0.92
PRECISION: 0.85
F1 SCORE: 0.83
FP RATE / ERROR I: 0.04
FN RATE / ERROR II: 0.19
________________________________________________________
MODEL PERFORMANCE ON VALIDATION DATA:



## False Positives (first 20):

In [15]:
display_images(false_positive_images[:20])

## False Negatives (first 20):

In [16]:
display_images(false_negative_images[:20])